# Autonomous driving - Traffic Light detection

In [541]:
import argparse
import os
import sys
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes, convert
from YAD2K.yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

%matplotlib inline

boxPath = './test/frameAnnotationsBOX.csv'
bulbPath = './test/frameAnnotationsBULB.csv'
img_path = './test/frames/'

In [542]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = .6):
    """ Filters YOLO boxes by thresholding on object and class confidence"""
    # Compute box scores
    box_scores = box_confidence * box_class_probs
    
    # Find the box_classes, keep track of the corresponding score
    box_classes = K.argmax(box_scores, axis=-1)
    box_class_scores = K.max(box_scores, axis=-1)
    
    # Create a filtering mask 
    filtering_mask = K.greater_equal(box_class_scores, threshold)
    
    # Apply the mask to scores, boxes and classes
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes,filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)
    
    return scores, boxes, classes

In [543]:
def iou(box1, box2):
    """ Intersection over union (IoU) between box1 and box2 """
    # Calculate (y1, x1, y2, x2)
    xi1 = np.maximum(box1[0], box2[0])
    yi1 = np.maximum(box1[1], box2[1])
    xi2 = np.minimum(box1[2], box2[2])
    yi2 = np.minimum(box1[3], box2[3])
    inter_area = max(xi2 - xi1, 0) * max(yi2 - yi1, 0)
    
    # Calculate Union area
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1]) 
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = (box1_area + box2_area) - inter_area
    
    # Compute IoU
    iou = inter_area / union_area
    
    return iou

In [544]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes=10, iou_threshold=0.5):
    """ apply non-max suppression to boxes"""
    
    max_boxes_tensor = K.variable(max_boxes, dtype='int32')
    K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
    
    # get list of indices corresponding to boxes we keep
    nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes_tensor, iou_threshold=iou_threshold)
    
    # select only nms_indices from scores, boxes, classes
    scores = K.gather(scores, nms_indices)
    boxes = K.gather(boxes, nms_indices)
    classes = K.gather(classes, nms_indices)
    
    return scores, boxes, classes

In [545]:
def yolo_eval(yolo_outputs, image_shape = (960., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    """ Converts the outputs of YOLO encoding """
    # retrieve outputs of YOLO model
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs
    
    # Convert boxes to be ready for filtering functions
    boxes = yolo_boxes_to_corners(box_xy, box_wh)
    
    # Perform score filtering
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold=score_threshold)
    
    # Scale boxes back to original image shape
    boxes = scale_boxes(boxes, image_shape)
    
    # Perform non-max suppression
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes=max_boxes, iou_threshold=iou_threshold)
    
    return scores, boxes, classes

# Test YOLO pretrained model

In [546]:
sess = K.get_session()

# define classes, anchors, image shape
class_names = read_classes("./YAD2K/model_data/classes.txt")
anchors = read_anchors("./YAD2K/model_data/yolo_anchors.txt")
image_shape = (960., 1280.)

# load pretrained model # model.save() instead of save_weights
yolo_model = load_model("./YAD2K/model_data/trained.h5")
yolo_model.summary()

# Convert model output to usable bounding box tensors
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

# Filter boxes
scores, boxes, classes = yolo_eval(yolo_outputs, image_shape)

C:\Program Files\Anaconda3\envs\tf\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 608, 608, 3)  0                                            
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 608, 608, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_23 (BatchNo (None, 608, 608, 32) 128         conv2d_25[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_23 (LeakyReLU)      (None, 608, 608, 32) 0           batch_normalization_23[0][0]     
__________________________________________________________________________________________________
max_poolin

In [547]:
def predict(sess, image_file, img_path=img_path):
    """ Runs the graph to predict boxes"""
    # Preproccess image
    image, image_data = preprocess_image(img_path + image_file, model_image_size=(608, 608))
    
    # run session with tensors
    out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes], feed_dict = {yolo_model.input: image_data, K.learning_phase(): 0})
    
    # Print predictions info
    print('Found {} boxes for {}'.format(len(out_boxes), image_file))
    # Generate colors for drawing bounding boxes.
    colors = generate_colors(class_names)
    # Draw bounding boxes on the image file
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    # Save the predicted bounding box on the image
    image.save(os.path.join("./test/out", image_file), quality=90)
    # Display the results in the notebook
    #output_image = scipy.misc.imread(os.path.join("out", image_file))
    #imshow(output_image)
    
    return out_scores, out_boxes, out_classes

In [548]:
def get_precision(sess, img, boxPath, bulbPath, n_classes=7):
    classes, true_classes, _ = get_classes(sess, img, boxPath, bulbPath)
    precision = np.divide(classes, true_classes)
    precision[np.isnan(precision)] = 0
    precision[np.isinf(precision)] = 0
    precision[np.isneginf(precision)] = 0
    return precision, true_classes

In [549]:
def avg_precision(sess, boxPath, bulbPath, img_path, n_classes=7):
    boxes = convert(boxPath, bulbPath)
    precision = np.zeros([n_classes])
    total = np.zeros([n_classes])
    for img in os.listdir(img_path):
        # calculate the precision for each image
        precision_img, true_classes = get_precision(sess, img, boxPath, bulbPath)
        precision += precision_img
        for i in range(len(total)):
            # if class is in the picture count up
            if true_classes[i] > 0.:
                total[i] += 1
    avg_precision = np.divide(precision, total)
    avg_precision[np.isnan(avg_precision)] = 0
    avg_precision[np.isinf(avg_precision)] = 0
    avg_precision[np.isneginf(avg_precision)] = 0
    return avg_precision, total

In [550]:
def mean_average_precision(sess, boxPath, bulbPath, img_path, n_classes=7):
    avg_pr, total = avg_precision(sess, boxPath, bulbPath, img_path)
    mAP = np.divide(np.sum(avg_pr), len(total[total > 0]))
    print("mAP Score: ", mAP)
    print(avg_pr)
    for i in range(len(avg_pr)):
        if avg_pr[i] > 0.:
            print("Average Precision for class", i, ": ", avg_pr[i])
    return mAP, avg_pr

In [551]:
# Count the number of correct predictions 
def build_cm(box, y_true, c, iou_threshold=0.5):
    # check against every box
    cm = np.zeros((n_classes, n_classes), dtype=np.int32)
    for index, row in y_true.iterrows():
        if iou(box, (row['Upper left corner Y'], row['Upper left corner X'],
                      row['Lower right corner Y'], row['Lower right corner X'])) > iou_threshold:
            cm[row['Annotation tag']][c] += 1
    return cm

In [552]:
def get_classes(sess, img, boxPath, bulbPath, c_classes=7, iou_threshold=0.5):
    # Concat the Boxes and Bulbs data 
    boxes = convert(boxPath, bulbPath)
    out_scores, out_boxes, out_classes = predict(sess, img)
    y_true = boxes[boxes['Filename'].str.match(img)]
    classes = np.zeros([n_classes])
    true_classes = np.zeros([n_classes])
    cm = np.zeros((n_classes, n_classes), dtype=np.int32)
    
    # get the number of correct predictions for each class
    for i, c in enumerate(out_classes):
        cm += build_cm(out_boxes[i], y_true, c, iou_threshold=iou_threshold)
        classes[c] += 1
   
    # get the number of true boxes per class
    for index, row in y_true.iterrows():
        true_classes[row['Annotation tag']] += 1
    
    print(out_classes)
    print("---->", int(np.sum(true_classes)), " boxes in total")
    print(true_classes)
    return classes, true_classes, cm

In [553]:
def get_cm(sess, cm, boxPath, bulbPath, img_path, n_classes=7, iou_threshold=0.5):
    boxes = convert(boxPath, bulbPath)
    classes = np.zeros([n_classes])
    true_classes = np.zeros([n_classes])
    cm = np.zeros((n_classes, n_classes), dtype=np.int32)
    i = 0
    for img in os.listdir(img_path):
        if i % 1 == 0:
            #get cm and classes from each image
            classes_n, true_classes_n, cm_n = get_classes(sess, img, boxPath, bulbPath, iou_threshold=iou_threshold)
            classes += classes_n
            true_classes += true_classes_n
            cm += cm_n
        i += 1
    return classes, true_classes, cm

In [554]:
#mAP, _ = mean_average_precision(sess, boxPath, bulbPath, img_path)

In [555]:
def get_prec_rec(sess, boxPath, bulbPath, img_path, n_classes=7, iou_threshold=0.5):
    cm = np.zeros((n_classes, n_classes), dtype=np.int32)
    classes, true_classes, cm = get_cm(sess, cm, boxPath, bulbPath, img_path, iou_threshold=iou_threshold)
    diag = np.diagonal(cm)
    precision = np.divide(diag.T, true_classes.T)
    recall = np.divide(diag, classes)
    precision[np.isnan(precision)] = 0
    precision[np.isinf(precision)] = 0
    precision[np.isneginf(precision)] = 0
    recall[np.isnan(recall)] = 0
    recall[np.isinf(recall)] = 0
    recall[np.isneginf(recall)] = 0
    return precision, recall, classes, true_classes, cm

In [556]:
t = 0.2
precision, recall, classes, true_classes, cm = get_prec_rec(sess, boxPath, bulbPath, img_path, iou_threshold=t)

Found 6 boxes for nightClip1--00000.jpg
[0 0 0 0 0 0]
----> 10  boxes in total
[10.  0.  0.  0.  0.  0.  0.]
Found 5 boxes for nightClip1--00001.jpg
[0 0 0 0 0]
----> 10  boxes in total
[10.  0.  0.  0.  0.  0.  0.]
Found 4 boxes for nightClip1--00002.jpg
[0 0 0 0]
----> 9  boxes in total
[9. 0. 0. 0. 0. 0. 0.]
Found 4 boxes for nightClip1--00003.jpg
[0 0 0 0]
----> 10  boxes in total
[10.  0.  0.  0.  0.  0.  0.]
Found 4 boxes for nightClip1--00004.jpg
[0 0 0 0]
----> 8  boxes in total
[8. 0. 0. 0. 0. 0. 0.]
Found 5 boxes for nightClip1--00005.jpg
[0 0 0 0 0]
----> 10  boxes in total
[10.  0.  0.  0.  0.  0.  0.]
Found 4 boxes for nightClip1--00006.jpg
[0 0 0 0]
----> 10  boxes in total
[10.  0.  0.  0.  0.  0.  0.]
Found 5 boxes for nightClip1--00007.jpg
[0 0 0 0 0]
----> 8  boxes in total
[8. 0. 0. 0. 0. 0. 0.]
Found 4 boxes for nightClip1--00008.jpg
[0 0 0 0]
----> 10  boxes in total
[10.  0.  0.  0.  0.  0.  0.]
Found 4 boxes for nightClip1--00009.jpg
[0 0 0 0]
----> 10  boxes in 

Found 7 boxes for nightClip1--00077.jpg
[0 0 0 0 0 0 0]
----> 12  boxes in total
[12.  0.  0.  0.  0.  0.  0.]
Found 7 boxes for nightClip1--00078.jpg
[0 0 0 0 0 0 0]
----> 12  boxes in total
[12.  0.  0.  0.  0.  0.  0.]
Found 6 boxes for nightClip1--00079.jpg
[0 0 0 0 0 0]
----> 11  boxes in total
[11.  0.  0.  0.  0.  0.  0.]
Found 6 boxes for nightClip1--00080.jpg
[0 0 0 0 0 0]
----> 11  boxes in total
[11.  0.  0.  0.  0.  0.  0.]
Found 7 boxes for nightClip1--00081.jpg
[0 0 0 0 0 0 0]
----> 12  boxes in total
[12.  0.  0.  0.  0.  0.  0.]
Found 7 boxes for nightClip1--00082.jpg
[0 0 0 0 0 0 0]
----> 12  boxes in total
[12.  0.  0.  0.  0.  0.  0.]
Found 7 boxes for nightClip1--00083.jpg
[0 0 0 0 0 0 0]
----> 11  boxes in total
[11.  0.  0.  0.  0.  0.  0.]
Found 7 boxes for nightClip1--00084.jpg
[0 0 0 0 0 0 0]
----> 11  boxes in total
[11.  0.  0.  0.  0.  0.  0.]
Found 6 boxes for nightClip1--00085.jpg
[0 0 0 0 0 0]
----> 11  boxes in total
[11.  0.  0.  0.  0.  0.  0.]
Found 5

Found 2 boxes for nightClip1--00154.jpg
[0 0]
----> 4  boxes in total
[4. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00155.jpg
[0 0]
----> 4  boxes in total
[4. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00156.jpg
[0 0]
----> 4  boxes in total
[4. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00157.jpg
[0 0]
----> 4  boxes in total
[4. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00158.jpg
[0 0]
----> 4  boxes in total
[4. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00159.jpg
[0 0]
----> 4  boxes in total
[4. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00160.jpg
[0 0]
----> 4  boxes in total
[4. 0. 0. 0. 0. 0. 0.]
Found 1 boxes for nightClip1--00161.jpg
[0]
----> 4  boxes in total
[4. 0. 0. 0. 0. 0. 0.]
Found 1 boxes for nightClip1--00162.jpg
[0]
----> 3  boxes in total
[3. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00163.jpg
[0 0]
----> 4  boxes in total
[4. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00164.jpg
[0 0]
----> 4  boxes in total
[4. 

Found 2 boxes for nightClip1--00242.jpg
[0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00243.jpg
[0 0]
----> 5  boxes in total
[5. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00244.jpg
[0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for nightClip1--00245.jpg
[0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00246.jpg
[0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00247.jpg
[0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00248.jpg
[0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00249.jpg
[0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00250.jpg
[0 0]
----> 4  boxes in total
[4. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00251.jpg
[0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00252.jpg
[0 0]
----> 6  boxes in tota

Found 6 boxes for nightClip1--00327.jpg
[0 0 0 0 0 0]
----> 8  boxes in total
[8. 0. 0. 0. 0. 0. 0.]
Found 4 boxes for nightClip1--00328.jpg
[0 0 0 0]
----> 7  boxes in total
[7. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for nightClip1--00329.jpg
[0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for nightClip1--00330.jpg
[0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for nightClip1--00331.jpg
[0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for nightClip1--00332.jpg
[0 0 0]
----> 5  boxes in total
[5. 0. 0. 0. 0. 0. 0.]
Found 4 boxes for nightClip1--00333.jpg
[0 0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 5 boxes for nightClip1--00334.jpg
[0 0 0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for nightClip1--00335.jpg
[0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for nightClip1--00336.jpg
[0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for nightClip1--00337.jp

Found 3 boxes for nightClip1--00412.jpg
[0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 4 boxes for nightClip1--00413.jpg
[0 0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for nightClip1--00414.jpg
[0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for nightClip1--00415.jpg
[0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for nightClip1--00416.jpg
[0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for nightClip1--00417.jpg
[0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 4 boxes for nightClip1--00418.jpg
[0 0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for nightClip1--00419.jpg
[0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 4 boxes for nightClip1--00420.jpg
[0 0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for nightClip1--00421.jpg
[0 0 0]
----> 6  boxes in total
[6. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for nightClip1--00422.jpg
[0 0

Found 0 boxes for nightClip1--00499.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for nightClip1--00500.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for nightClip1--00501.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for nightClip1--00502.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for nightClip1--00503.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for nightClip1--00504.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for nightClip1--00505.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for nightClip1--00506.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for nightClip1--00507.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for nightClip1--00508.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for nightClip1--00509.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 bo

C:\Program Files\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """
C:\Program Files\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


In [557]:
print(precision)
print(recall)
print(classes)
print(true_classes)
print(cm)

[0.65518192 0.         0.         0.         0.         0.
 0.        ]
[1.20659898 0.         0.         0.         0.         0.
 0.        ]
[1970.    0.    2.    0.    0.    0.    0.]
[3628.    0.    0.    0.    0.    0.    0.]
[[2377    0    3    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]]
